<a href="https://colab.research.google.com/github/shrenikraxit/T-100-Analysis/blob/main/T100_Data_Prep_v4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Concatenate the T-100 files

Year = 2018, 2019, 2023 and 2024

In [ ]:
import pandas as pd
import numpy as np

# Step 0 - Environment preparation

# Connect with google drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
from pathlib import Path

def consolidate_t100_simple(data_directory, years=None):

    print(f"🔍 Looking for T-100 files in: {data_directory}")

    # Find all T-100 files
    pattern = str(Path(data_directory) / "T_T100_MARKET_ALL_CARRIER *.csv")
    files = sorted(glob.glob(pattern))

    if not files:
        print(f"❌ No files found matching: T_T100_MARKET_ALL_CARRIER *.csv")
        print(f"Files in directory: {list(Path(data_directory).glob('*.csv'))}")
        return None

    # Filter by years if specified
    if years:
        files = [f for f in files if any(str(year) in Path(f).name for year in years)]

    print(f"📁 Found {len(files)} files:")
    for file in files:
        size_mb = Path(file).stat().st_size / (1024**2)
        print(f"   • {Path(file).name} ({size_mb:.1f} MB)")

    # Read and combine all files
    print(f"\n📊 Reading and combining files...")

    all_dataframes = []
    total_rows = 0

    for file in files:
        print(f"   Reading {Path(file).name}...", end=" ")

        # Read file in chunks to handle large files
        chunks = []
        for chunk in pd.read_csv(file, chunksize=50000):
            chunks.append(chunk)

        df = pd.concat(chunks, ignore_index=True)
        all_dataframes.append(df)

        rows = len(df)
        total_rows += rows
        print(f"{rows:,} rows")

    # Combine all dataframes
    print(f"\n🔗 Combining {len(all_dataframes)} files...")
    combined_data = pd.concat(all_dataframes, ignore_index=True)

    # Basic info
    print(f"✅ Combined dataset:")
    print(f"   • Total rows: {len(combined_data):,}")
    print(f"   • Total columns: {len(combined_data.columns)}")
    if 'YEAR' in combined_data.columns:
        years_found = sorted(combined_data['YEAR'].unique())
        print(f"   • Years: {years_found}")
    if 'PASSENGERS' in combined_data.columns:
        total_pax = combined_data['PASSENGERS'].sum()
        print(f"   • Total passengers: {total_pax:,.0f}")

    # Save consolidated file
    output_file = Path(data_directory) / "consolidated_t100_data.csv"
    print(f"\n💾 Saving to: {output_file}")

    combined_data.to_csv(output_file, index=False)

    size_mb = output_file.stat().st_size / (1024**2)
    print(f"✅ Saved! File size: {size_mb:.1f} MB")

    return str(output_file)

# ===== USAGE =====

# UPDATE THIS PATH:
DATA_DIRECTORY = "/content/drive/MyDrive/airline_data_analysis_v2/"  # <-- CHANGE THIS!

# OPTION 1: Consolidate all years
consolidated_file = consolidate_t100_simple(DATA_DIRECTORY, years=[2018, 2019, 2023, 2024])

# Print result
if consolidated_file:
    print(f"\n🎉 SUCCESS!")
    print(f"   Consolidated file: {consolidated_file}")
    print(f"\n📋 Next step - use in your seasonal analysis:")
    print(f'   filepath = "{consolidated_file}"')
    print(f'   results = run_seasonal_analysis(filepath, years=[2018, 2019, 2023, 2024]')
else:
    print(f"\n❌ Failed. Please check your directory path.")

# If you just want to test with one year:
# test_file = consolidate_t100_simple(DATA_DIRECTORY, years=[2019])  # Just 2019 for testing

🔍 Looking for T-100 files in: /content/drive/MyDrive/airline_data_analysis_v2/
📁 Found 4 files:
   • T_T100_MARKET_ALL_CARRIER 2018.csv (80.8 MB)
   • T_T100_MARKET_ALL_CARRIER 2019.csv (80.6 MB)
   • T_T100_MARKET_ALL_CARRIER 2023.csv (82.6 MB)
   • T_T100_MARKET_ALL_CARRIER 2024.csv (80.6 MB)

📊 Reading and combining files...
   Reading T_T100_MARKET_ALL_CARRIER 2018.csv... 321,478 rows
   Reading T_T100_MARKET_ALL_CARRIER 2019.csv... 321,388 rows
   Reading T_T100_MARKET_ALL_CARRIER 2023.csv... 327,440 rows
   Reading T_T100_MARKET_ALL_CARRIER 2024.csv... 320,910 rows

🔗 Combining 4 files...
✅ Combined dataset:
   • Total rows: 1,291,216
   • Total columns: 41
   • Years: [np.int64(2018), np.int64(2019), np.int64(2023), np.int64(2024)]
   • Total passengers: 4,242,158,055

💾 Saving to: /content/drive/MyDrive/airline_data_analysis_v2/consolidated_t100_data.csv
✅ Saved! File size: 327.2 MB

🎉 SUCCESS!
   Consolidated file: /content/drive/MyDrive/airline_data_analysis_v2/consolidated_t

# Step 2
T-100 Data Cleanup Script
Comprehensive data quality checks and cleanup for consolidated_t100_data.csv

This script performs the following cleanup operations:
1. Remove cargo-only flights (PASSENGERS = 0)
2. Check for missing/anomalous passenger counts
3. Remove duplicate route entries
4. Filter for domestic U.S. routes only
5. Handle seasonal variations (quarterly aggregation)

In [ ]:
#!/usr/bin/env python3
"""
T-100 Data Cleanup Script
Comprehensive data quality checks and cleanup for consolidated_t100_data.csv

This script performs the following cleanup operations:
1. Remove cargo-only flights (PASSENGERS = 0)
2. Check for missing/anomalous passenger counts
3. Filter for domestic U.S. routes only
4. Remove duplicate route entries
5. Handle seasonal variations (quarterly aggregation)

NOTE: Airport-code validation has been removed per request.
"""

import pandas as pd
import numpy as np
import logging
from datetime import datetime
import warnings
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up data directory path
DATA_DIRECTORY = "/content/drive/MyDrive/airline_data_analysis_v2/"

# Ensure directory exists
os.makedirs(DATA_DIRECTORY, exist_ok=True)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'{DATA_DIRECTORY}t100_cleanup.log'),
        logging.StreamHandler()
    ]
)

def setup_cleanup_summary():
    """Initialize cleanup summary tracking"""
    return {
        'original_rows': 0,
        'cargo_only_removed': 0,
        'missing_passengers_removed': 0,
        'anomalous_passengers_removed': 0,
        'non_domestic_removed': 0,
        'duplicates_removed': 0,
        'final_rows': 0,
        'data_quality_issues': []
    }

def load_t100_data(filepath):
    """Load T-100 data with memory optimization"""
    logging.info(f"Loading T-100 data from {filepath}")

    try:
        # Read in chunks to handle large files
        chunk_list = []
        chunk_size = 100000  # Adjust based on available memory

        for chunk in pd.read_csv(filepath, chunksize=chunk_size, low_memory=False):
            chunk_list.append(chunk)

        df = pd.concat(chunk_list, ignore_index=True)

        logging.info(f"Successfully loaded {len(df):,} rows with {len(df.columns)} columns")
        logging.info(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

        return df

    except Exception as e:
        logging.error(f"Error loading T-100 data: {e}")
        raise

def cleanup_cargo_only_flights(df, summary):
    """Remove cargo-only flights (PASSENGERS = 0)"""
    logging.info("Step 1: Removing cargo-only flights...")

    initial_count = len(df)

    # Remove rows where PASSENGERS is 0 or NaN
    cargo_mask = (df['PASSENGERS'] == 0) | (df['PASSENGERS'].isna())
    cargo_count = cargo_mask.sum()

    df_clean = df[~cargo_mask].copy()

    summary['cargo_only_removed'] = cargo_count
    logging.info(f"Removed {cargo_count:,} cargo-only flights ({cargo_count/initial_count*100:.1f}%)")
    logging.info(f"Remaining rows: {len(df_clean):,}")

    return df_clean

def check_passenger_data_quality(df, summary):
    """Check for missing and anomalous passenger counts"""
    logging.info("Step 2: Checking passenger data quality...")

    initial_count = len(df)

    # Check for missing passenger data
    missing_passengers = df['PASSENGERS'].isna().sum()
    if missing_passengers > 0:
        logging.warning(f"Found {missing_passengers:,} rows with missing passenger data")
        df = df.dropna(subset=['PASSENGERS'])
        summary['missing_passengers_removed'] = missing_passengers

    # Check for negative passenger counts
    negative_passengers = (df['PASSENGERS'] < 0).sum()
    if negative_passengers > 0:
        logging.warning(f"Found {negative_passengers:,} rows with negative passenger counts")
        df = df[df['PASSENGERS'] >= 0]
        summary['anomalous_passengers_removed'] += negative_passengers

    # Check for extremely high passenger counts (potential data errors)
    passenger_threshold = df['PASSENGERS'].quantile(0.999)
    extreme_passengers = (df['PASSENGERS'] > passenger_threshold).sum()

    if extreme_passengers > 0:
        logging.info(f"Found {extreme_passengers:,} rows with extremely high passenger counts (>{passenger_threshold:,.0f})")
        logging.info("These will be flagged but not removed (could be legitimate large aircraft)")
        summary['data_quality_issues'].append(f"Extreme passenger counts: {extreme_passengers:,} rows")

    # Passenger statistics
    passenger_stats = df['PASSENGERS'].describe()
    logging.info("Passenger count statistics:")
    logging.info(f"  Mean: {passenger_stats['mean']:,.0f}")
    logging.info(f"  Median: {passenger_stats['50%']:,.0f}")
    logging.info(f"  Max: {passenger_stats['max']:,.0f}")

    removed_count = initial_count - len(df)
    if removed_count > 0:
        logging.info(f"Removed {removed_count:,} rows with data quality issues")

    return df

def filter_domestic_routes(df, summary):
    """Filter for domestic U.S. routes only"""
    logging.info("Step 3: Filtering for domestic U.S. routes...")

    initial_count = len(df)

    # Check available country codes
    if 'ORIGIN_COUNTRY' not in df.columns or 'DEST_COUNTRY' not in df.columns:
        raise KeyError("Expected columns 'ORIGIN_COUNTRY' and 'DEST_COUNTRY' not found in dataset.")

    origin_countries = df['ORIGIN_COUNTRY'].value_counts()
    dest_countries = df['DEST_COUNTRY'].value_counts()

    logging.info(f"Origin countries found: {list(origin_countries.index)}")
    logging.info(f"Destination countries found: {list(dest_countries.index)}")

    # Filter for domestic routes (both origin and destination in US)
    domestic_mask = (df['ORIGIN_COUNTRY'] == 'US') & (df['DEST_COUNTRY'] == 'US')
    df_domestic = df[domestic_mask].copy()

    removed_count = initial_count - len(df_domestic)
    summary['non_domestic_removed'] = removed_count

    logging.info(f"Removed {removed_count:,} non-domestic routes ({removed_count/initial_count*100:.1f}%)")
    logging.info(f"Remaining domestic routes: {len(df_domestic):,}")

    return df_domestic

def remove_duplicates(df, summary):
    """Remove duplicate route entries"""
    logging.info("Step 4: Removing duplicate entries...")

    initial_count = len(df)

    # Define key columns that identify a unique route-time combination
    key_columns = [
        'YEAR', 'QUARTER', 'MONTH',
        'UNIQUE_CARRIER', 'ORIGIN', 'DEST',
        'CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'
    ]

    existing_key_columns = [col for col in key_columns if col in df.columns]
    logging.info(f"Using columns for duplicate detection: {existing_key_columns}")

    duplicate_mask = df.duplicated(subset=existing_key_columns, keep='first')
    duplicate_count = duplicate_mask.sum()

    if duplicate_count > 0:
        logging.warning(f"Found {duplicate_count:,} duplicate entries")
        sample_duplicates = df[duplicate_mask].head()
        logging.info("Sample duplicate entries:")
        for _, row in sample_duplicates.iterrows():
            try:
                logging.info(f"  {row.get('YEAR','?')}-Q{row.get('QUARTER','?')} {row.get('ORIGIN','?')}->{row.get('DEST','?')} {row.get('UNIQUE_CARRIER','?')}")
            except Exception:
                pass

    df_clean = df[~duplicate_mask].copy()

    summary['duplicates_removed'] = duplicate_count
    logging.info(f"Removed {duplicate_count:,} duplicate entries")
    logging.info(f"Remaining rows: {len(df_clean):,}")

    return df_clean

def handle_seasonal_variations(df):
    """Aggregate data quarterly and add seasonal analysis"""
    logging.info("Step 5: Handling seasonal variations...")

    # Ensure QUARTER column exists and is valid
    if 'QUARTER' in df.columns:
        valid_quarters = df['QUARTER'].between(1, 4)
        invalid_quarters = (~valid_quarters).sum()

        if invalid_quarters > 0:
            logging.warning(f"Found {invalid_quarters:,} rows with invalid quarter values")
            df = df[valid_quarters]

    # Create quarterly aggregation
    group_cols = ['YEAR', 'QUARTER', 'ORIGIN', 'DEST']
    agg_map = {
        'PASSENGERS': 'sum',
        'FREIGHT': 'sum',
        'MAIL': 'sum',
        'DISTANCE': 'mean',
    }
    # Add optional columns if present
    optional_first = ['UNIQUE_CARRIER', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST_STATE_ABR']
    for col in optional_first:
        if col in df.columns:
            agg_map[col] = 'first'

    quarterly_agg = df.groupby([c for c in group_cols if c in df.columns]).agg(agg_map).reset_index()

    logging.info(f"Quarterly aggregation created: {len(quarterly_agg):,} route-quarter combinations")

    # Calculate seasonal patterns
    if 'QUARTER' in quarterly_agg.columns and 'PASSENGERS' in quarterly_agg.columns:
        seasonal_stats = quarterly_agg.groupby('QUARTER')['PASSENGERS'].agg(['sum', 'mean', 'count'])
        logging.info("Seasonal passenger patterns:")
        for quarter in [1, 2, 3, 4]:
            if quarter in seasonal_stats.index:
                stats = seasonal_stats.loc[quarter]
                logging.info(f"  Q{quarter}: {stats['sum']:,} total passengers, {stats['mean']:,.0f} avg per route")

    return df, quarterly_agg

def generate_cleanup_report(summary, output_dir='.'):
    """Generate comprehensive cleanup report"""
    logging.info("Generating cleanup report...")

    # Calculate percentages
    total_removed = (summary['cargo_only_removed'] +
                     summary['missing_passengers_removed'] +
                     summary['anomalous_passengers_removed'] +
                     summary['non_domestic_removed'] +
                     summary['duplicates_removed'])

    reduction_rate = (total_removed / summary['original_rows']) * 100 if summary['original_rows'] > 0 else 0

    # Create report
    report = f"""
T-100 DATA CLEANUP REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

SUMMARY:
========
Original dataset size:        {summary['original_rows']:,} rows
Final dataset size:           {summary['final_rows']:,} rows
Total rows removed:           {total_removed:,} rows
Data reduction:               {reduction_rate:.1f}%

CLEANUP OPERATIONS:
==================
1. Cargo-only flights removed:       {summary['cargo_only_removed']:,} rows
2. Missing passenger data removed:   {summary['missing_passengers_removed']:,} rows
3. Anomalous passenger data removed: {summary['anomalous_passengers_removed']:,} rows
4. Non-domestic routes removed:      {summary['non_domestic_removed']:,} rows
5. Duplicate entries removed:        {summary['duplicates_removed']:,} rows

DATA QUALITY ISSUES IDENTIFIED:
==============================
"""
    for issue in summary['data_quality_issues']:
        report += f"- {issue}\n"

    report_path = f"{output_dir}/t100_cleanup_report.txt"
    with open(report_path, 'w') as f:
        f.write(report)

    logging.info(f"Cleanup report saved to: {report_path}")
    print(report)

def main():
    """Main cleanup execution function"""
    start_time = datetime.now()
    logging.info("Starting T-100 data cleanup process...")

    # Initialize summary
    summary = setup_cleanup_summary()

    try:
        # File paths - Updated to use specified directory
        t100_file = f"{DATA_DIRECTORY}consolidated_t100_data.csv"
        output_dir = DATA_DIRECTORY

        logging.info(f"Data directory: {DATA_DIRECTORY}")
        logging.info(f"T-100 file: {t100_file}")

        # Check if required file exists
        if not os.path.exists(t100_file):
            raise FileNotFoundError(f"T-100 data file not found: {t100_file}")

        logging.info("✅ Required file found")

        # Load T-100 data
        df = load_t100_data(t100_file)
        summary['original_rows'] = len(df)

        logging.info(f"Original data columns: {list(df.columns)}")

        # Cleanup pipeline (airport-code validation intentionally skipped)
        df = cleanup_cargo_only_flights(df, summary)
        df = check_passenger_data_quality(df, summary)
        df = filter_domestic_routes(df, summary)
        df = remove_duplicates(df, summary)

        # Seasonal handling / quarterly aggregation
        df_clean, quarterly_agg = handle_seasonal_variations(df)

        summary['final_rows'] = len(df_clean)

        # Save cleaned data
        output_file = f"{output_dir}consolidated_t100_data_cleaned.csv"
        quarterly_file = f"{output_dir}t100_quarterly_aggregated.csv"

        logging.info(f"Saving cleaned data to {output_file}...")
        df_clean.to_csv(output_file, index=False)

        logging.info(f"Saving quarterly aggregated data to {quarterly_file}...")
        quarterly_agg.to_csv(quarterly_file, index=False)

        # Generate report
        generate_cleanup_report(summary, output_dir)

        # Final statistics
        end_time = datetime.now()
        duration = end_time - start_time

        logging.info(f"Cleanup completed successfully in {duration}")
        logging.info(f"Clean dataset: {len(df_clean):,} rows")
        logging.info(f"Quarterly aggregated dataset: {len(quarterly_agg):,} rows")

        return df_clean, quarterly_agg, summary

    except Exception as e:
        logging.error(f"Error during cleanup process: {e}")
        raise

if __name__ == "__main__":
    # Run the cleanup process
    cleaned_data, quarterly_data, cleanup_summary = main()

    print("\n" + "="*60)
    print("T-100 DATA CLEANUP COMPLETED SUCCESSFULLY!")
    print("="*60)
    print(f"Files created in /content/drive/MyDrive/airline_data_analysis_v2/:")
    print(f"- consolidated_t100_data_cleaned.csv")
    print(f"- t100_quarterly_aggregated.csv")
    print(f"- t100_cleanup_report.txt")
    print(f"- t100_cleanup.log")


T-100 DATA CLEANUP REPORT
Generated: 2025-08-29 21:18:51

SUMMARY:
Original dataset size:        1,291,216 rows
Final dataset size:           815,641 rows
Total rows removed:           475,575 rows
Data reduction:               36.8%

CLEANUP OPERATIONS:
1. Cargo-only flights removed:       229,336 rows
2. Missing passenger data removed:   0 rows
3. Anomalous passenger data removed: 0 rows
4. Non-domestic routes removed:      228,340 rows
5. Duplicate entries removed:        17,899 rows

DATA QUALITY ISSUES IDENTIFIED:
- Extreme passenger counts: 1,061 rows


T-100 DATA CLEANUP COMPLETED SUCCESSFULLY!
Files created in /content/drive/MyDrive/airline_data_analysis_v2/:
- consolidated_t100_data_cleaned.csv
- t100_quarterly_aggregated.csv
- t100_cleanup_report.txt
- t100_cleanup.log


# Step 3 - Extract all US airports from the T-100 dataset
include all origin and destination airports

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIRECTORY = "/content/drive/MyDrive/airline_data_analysis_v2/"  # <-- CHANGE THIS IF NEEDED
consolidated_file = Path(DATA_DIRECTORY) / "consolidated_t100_data_cleaned.csv"
unique_airports_t100 = Path(DATA_DIRECTORY) / "unique_airports_t100.csv"

# Load the CSV
df = pd.read_csv(consolidated_file, dtype=str)  # keep as string to avoid type issues

# Helper to pick the first matching column name if present
def pick(*names):
    return next((c for c in names if c in df.columns), None)

# Sanity check for core columns
if 'ORIGIN' not in df.columns or 'DEST' not in df.columns:
    raise KeyError("Expected columns 'ORIGIN' and 'DEST' not found in the cleaned T-100 file.")

# Candidate columns (some datasets have slightly different names)
origin_id_col   = pick('ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_ID')
origin_city_col = pick('ORIGIN_CITY_NAME', 'ORIGIN_CITY')
origin_ctry_col = pick('ORIGIN_COUNTRY')

dest_id_col     = pick('DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_ID')
dest_city_col   = pick('DEST_CITY_NAME', 'DEST_CITY')
dest_ctry_col   = pick('DEST_COUNTRY')

# Build origin airports frame
origin_rename = {
    **({origin_id_col: 'AIRPORT_ID'} if origin_id_col else {}),
    'ORIGIN': 'AIRPORT_NAME',
    **({origin_city_col: 'CITY'} if origin_city_col else {}),
    **({origin_ctry_col: 'COUNTRY'} if origin_ctry_col else {}),
}
origin_airports = df[list(origin_rename.keys())].rename(columns=origin_rename)

# Build destination airports frame
dest_rename = {
    **({dest_id_col: 'AIRPORT_ID'} if dest_id_col else {}),
    'DEST': 'AIRPORT_NAME',
    **({dest_city_col: 'CITY'} if dest_city_col else {}),
    **({dest_ctry_col: 'COUNTRY'} if dest_ctry_col else {}),
}
dest_airports = df[list(dest_rename.keys())].rename(columns=dest_rename)

# Combine, normalize, and dedupe
all_airports = pd.concat([origin_airports, dest_airports], ignore_index=True)

# Clean up whitespace/case for safety
for col in ['AIRPORT_ID', 'AIRPORT_NAME', 'CITY', 'COUNTRY']:
    if col in all_airports.columns:
        all_airports[col] = all_airports[col].astype(str).str.strip()

# Prefer uppercase codes for AIRPORT_NAME if they’re IATA/FAA codes
if 'AIRPORT_NAME' in all_airports.columns:
    all_airports['AIRPORT_NAME'] = all_airports['AIRPORT_NAME'].str.upper()

# Create a dedup key: use AIRPORT_ID if available, else fall back to AIRPORT_NAME
dedup_key = all_airports['AIRPORT_ID'].where(all_airports['AIRPORT_ID'].notna() & (all_airports['AIRPORT_ID'] != ''), all_airports.get('AIRPORT_NAME'))
all_airports['__DEDUP_KEY__'] = dedup_key

# Drop duplicates on the key
all_airports = all_airports.drop_duplicates(subset='__DEDUP_KEY__').drop(columns='__DEDUP_KEY__')

# Sort (by ID if present, else by name)
if 'AIRPORT_ID' in all_airports.columns:
    all_airports = all_airports.sort_values(by=['AIRPORT_ID', 'AIRPORT_NAME'])
else:
    all_airports = all_airports.sort_values(by=['AIRPORT_NAME'])

# Optional: filter to US only (the cleaned file should already be domestic-only, so usually unnecessary)
# if 'COUNTRY' in all_airports.columns:
#     all_airports = all_airports[all_airports['COUNTRY'].str.upper() == 'US']

# Reset index
all_airports = all_airports.reset_index(drop=True)

# Save to CSV
all_airports.to_csv(unique_airports_t100, index=False)

print(f"Total unique airports found: {len(all_airports)}")
print(all_airports.head())


Total unique airports found: 1636
  AIRPORT_ID AIRPORT_NAME              CITY COUNTRY
0      10001          01A  Afognak Lake, AK      US
1      10005          05A  Little Squaw, AK      US
2      10006          06A      Kizhuyak, AK      US
3      10009          09A         Homer, AK      US
4      10010          1B1        Hudson, NY      US


# Step 3.A - Prepare for Step 5
- Find the unique list of cities and state combination from the unique_airports_t100.csv
- Check how many are not present in the file "city_business_leisure_classification_detailed.csv"

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIRECTORY = "/content/drive/MyDrive/airline_data_analysis_v2/"
unique_airports_file = Path(DATA_DIRECTORY) / "unique_airports_t100.csv"
city_classification_file = Path(DATA_DIRECTORY) / "city_business_leisure_classification_detailed.csv"
out_csv = Path(DATA_DIRECTORY) / "unmatched_city_state_combos.csv"

# Load
df_airports = pd.read_csv(unique_airports_file, dtype=str)
df_class = pd.read_csv(city_classification_file, dtype=str)

# Split CITY into City / State (robust to missing comma)
split = df_airports["CITY"].fillna("").str.rsplit(",", n=1, expand=True)
df_airports["City"] = split[0].str.strip()
df_airports["State"] = split[1].str.strip()

# Normalize for join
df_airports["City_norm"] = df_airports["City"].str.lower()
df_airports["State_norm"] = df_airports["State"].str.lower()
df_class["City_norm"] = df_class["City"].astype(str).str.strip().str.lower()
df_class["State_norm"] = df_class["State/Territory"].astype(str).str.strip().str.lower()

# Left join to find city+state in airports not present in classification
merged = df_airports.merge(
    df_class[["City_norm", "State_norm"]].drop_duplicates(),
    on=["City_norm", "State_norm"],
    how="left",
    indicator=True
)

# Unique unmatched combos
unmatched = (merged[merged["_merge"] == "left_only"]
             [["City", "State"]]
             .dropna()
             .drop_duplicates()
             .sort_values(["State", "City"]))

print(f"Number of unmatched City+State combinations: {len(unmatched)}")

# Save
unmatched.to_csv(out_csv, index=False)
print(f"Saved unmatched combinations to: {out_csv}")


Number of unmatched City+State combinations: 0
Saved unmatched combinations to: /content/drive/MyDrive/airline_data_analysis_v2/unmatched_city_state_combos.csv


# Step 4 - Hub Status from 2018

- To be used for the Hypothesis 1


In [ ]:
import pandas as pd
from pathlib import Path

# ========= CONFIG =========
DATA_DIRECTORY = "/content/drive/MyDrive/airline_data_analysis_v2/"  # <-- CHANGE IF NEEDED
cleaned_t100_path = Path(DATA_DIRECTORY) / "consolidated_t100_data_cleaned.csv"
airports_in_path  = Path(DATA_DIRECTORY) / "unique_airports_t100.csv"
airports_out_path = Path(DATA_DIRECTORY) / "unique_airports_t100_hub_updated.csv"
HUB_YEAR = 2018
# =========================

def classify_airport_hub(passengers, total_us_passengers):
    """User-provided logic: classify by share of total US passengers (%)."""
    if total_us_passengers <= 0 or pd.isna(passengers):
        return "Non-Hub Primary"
    share = (passengers / total_us_passengers) * 100.0
    if share >= 1.0:
        return "Major Hub"
    elif share >= 0.25:
        return "Medium Hub"
    elif share >= 0.05:
        return "Small Hub"
    else:
        return "Non-Hub Primary"

def pick(df, *names):
    """Return the first existing column name from candidates."""
    return next((c for c in names if c in df.columns), None)

# ---------- Load data ----------
df = pd.read_csv(cleaned_t100_path, dtype=str)
airports_df = pd.read_csv(airports_in_path, dtype=str)

# Basic checks
if 'YEAR' not in df.columns or 'PASSENGERS' not in df.columns:
    raise KeyError("Expected columns 'YEAR' and 'PASSENGERS' in the cleaned T-100 file.")

# Convert types
df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce').astype('Int64')
df['PASSENGERS'] = pd.to_numeric(df['PASSENGERS'], errors='coerce')

# Filter to the requested year
df_2018 = df[df['YEAR'] == HUB_YEAR].copy()

# Identify airport key columns (prefer IDs; fall back to codes)
origin_id_col = pick(df_2018, 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_ID')
dest_id_col   = pick(df_2018, 'DEST_AIRPORT_ID',   'DEST_AIRPORT_SEQ_ID',   'DEST_ID')
origin_col    = pick(df_2018, 'ORIGIN')
dest_col      = pick(df_2018, 'DEST')

if not origin_col or not dest_col:
    raise KeyError("Expected 'ORIGIN' and 'DEST' columns in the cleaned T-100 file.")

# Build per-airport passenger totals for 2018 (counting both origin and destination)
def total_by_key(key_col):
    return (
        df_2018.groupby(key_col, dropna=True)['PASSENGERS']
        .sum(min_count=1)
        .rename('PAX_2018')
        .astype(float)
    )

if origin_id_col and dest_id_col:
    origin_tot = total_by_key(origin_id_col)
    dest_tot   = total_by_key(dest_id_col)
    pax_by_airport = origin_tot.add(dest_tot, fill_value=0.0)
    pax_by_airport.index.name = 'AIRPORT_ID'
    key_for_merge = 'AIRPORT_ID'
    airports_df['AIRPORT_ID'] = airports_df.get('AIRPORT_ID', '').astype(str).str.strip()
    pax_df = pax_by_airport.reset_index()
    pax_df['AIRPORT_ID'] = pax_df['AIRPORT_ID'].astype(str).str.strip()
else:
    origin_tot = total_by_key(origin_col)
    dest_tot   = total_by_key(dest_col)
    pax_by_airport = origin_tot.add(dest_tot, fill_value=0.0)
    pax_by_airport.index.name = 'AIRPORT_NAME'
    key_for_merge = 'AIRPORT_NAME'
    if 'AIRPORT_NAME' not in airports_df.columns:
        raise KeyError("Fallback requires 'AIRPORT_NAME' in unique_airports_t100.csv.")
    airports_df['AIRPORT_NAME'] = airports_df['AIRPORT_NAME'].astype(str).str.upper().str.strip()
    pax_df = pax_by_airport.reset_index()
    pax_df['AIRPORT_NAME'] = pax_df['AIRPORT_NAME'].astype(str).str.upper().str.strip()

# National total and shares
total_us_pax_2018 = pax_df['PAX_2018'].sum()
pax_df['US_SHARE_2018_PCT'] = (pax_df['PAX_2018'] / total_us_pax_2018 * 100.0).round(6)
pax_df['HUB_TYPE'] = pax_df['PAX_2018'].apply(lambda x: classify_airport_hub(x, total_us_pax_2018))
pax_df['HUB_YEAR'] = HUB_YEAR

# Merge into the airports table
cols_to_add = ['PAX_2018', 'US_SHARE_2018_PCT', 'HUB_TYPE', 'HUB_YEAR']
merged = airports_df.merge(pax_df[[key_for_merge] + cols_to_add], on=key_for_merge, how='left')

# >>> NEW: mark airports not present in 2018 as NEW in HUB_TYPE
merged['HUB_TYPE'] = merged['HUB_TYPE'].fillna('NEW')

# (Optional) If you also want to zero-fill metrics for NEW airports, uncomment:
merged['PAX_2018'] = merged['PAX_2018'].fillna(0.0)
merged['US_SHARE_2018_PCT'] = merged['US_SHARE_2018_PCT'].fillna(0.0)

# Sort & save
sort_cols = [c for c in ['AIRPORT_ID', 'AIRPORT_NAME'] if c in merged.columns]
if sort_cols:
    merged = merged.sort_values(by=sort_cols).reset_index(drop=True)

merged.to_csv(airports_out_path, index=False)

print(f"✅ Hub classification completed for {HUB_YEAR}.")
print(f"   Total US passengers counted (both ends): {total_us_pax_2018:,.0f}")
print("   Hub-type counts (including NEW):")
print(merged['HUB_TYPE'].value_counts(dropna=False))
print(f"\n💾 Saved: {airports_out_path}")


✅ Hub classification completed for 2018.
   Total US passengers counted (both ends): 1,518,702,636
   Hub-type counts (including NEW):
HUB_TYPE
Non-Hub Primary    1050
NEW                 446
Small Hub            71
Medium Hub           40
Major Hub            29
Name: count, dtype: int64

💾 Saved: /content/drive/MyDrive/airline_data_analysis_v2/unique_airports_t100_hub_updated.csv


# Step 5 - creates "consolidated_t100_data_cleaned_tp_updated.csv" by assigning travel purpose classifications to every row in the consolidated T-100 dataset

- To be used for the Hypothesis 2


#Step 5 (Alternative)

In [ ]:
# compute_tpi_for_t100_city_state.py  (CITY_STATE-based matching + diagnostics)
from pathlib import Path
import re, time
import pandas as pd
import numpy as np
from collections import Counter

# ========= File locations =========
DATA_DIRECTORY = Path("/content/drive/MyDrive/airline_data_analysis_v2/")
T100_PATH   = DATA_DIRECTORY / "consolidated_t100_data_cleaned.csv"
CLASS_PATH  = DATA_DIRECTORY / "city_business_leisure_classification_detailed.csv"
OUT_PATH    = DATA_DIRECTORY / "consolidated_t100_with_tpi_city_state.csv"
MISSING_OUT = DATA_DIRECTORY / "missing_consolidated_t100_with_tpi_city_state.csv"

# Diagnostics outputs
UNMATCHED_ORIG_COUNTS = DATA_DIRECTORY / "unmatched_origin_city_state_counts.csv"
UNMATCHED_DEST_COUNTS = DATA_DIRECTORY / "unmatched_dest_city_state_counts.csv"

# ========= Batch settings =========
CHUNK_SIZE = 250_000

# ========= Model settings =========
DESTINATION_WEIGHT = 0.6
THRESHOLDS = (40.0, 60.0)
ORIG_MULT = 0.8
DEST_MULT = 1.0
DEST_SEASON_BONUS = 1.1
CONF_WEIGHTS = (0.6, 0.4)

MARKET_DELTAS = { "Market_Military/Defense_Base": +12, "Market_Military_Test/Eval_Range": +10, "Market_StateCapital/Government": +8, "Market_NorthSlope_Oilfield": +15, "Market_Mining": +12, "Market_Remote/EAS": +10, "Market_Fishing/Cannery": +8, "Market_AlaskaRegionalHub": +5, "Market_DiversifiedMetro/Corporate": +5, "Market_RegionalServices/Healthcare/LightIndustry": +4, "Market_Bizjet_Hub": +8, "Market_Border/Port_Logistics": +6, "Market_Port/Shipbuilding/Shipyards": +5, "Market_Port/Coastal_Mixed": +2, "Market_CruiseGateway/Port": -8, "Market_ParkGateway": -8, "Market_WildernessLodge/Adventure": -10, "Market_ResortIsland": -12, "Market_CoastalTourism/Golf/Aquarium": -6, "Market_Wine/Culinary": -6, "Market_Weekender/HistoricTown": -4, "Market_Ski/Outdoor_Gateway": -6, "Market_Ferry_Gateway": -3, "Market_Island/Interisland_Essential": -3, }
LEXICAL_NEG_DELTAS = { "Market_LexicalCue_Springs": -2, "Market_LexicalCue_Lake": -2, "Market_LexicalCue_Valley": -2, "Market_LexicalCue_Park": -2, "Market_LexicalCue_Ocean": -2, "Market_LexicalCue_Harbor": -2, "Market_LexicalCue_Island": -2, }
LEXICAL_CAP = -5.0
SEASON_DELTAS = { "Season_SummerPeaks": -6, "Season_WeekendHeavy": -4, "Season_Festivals/Events": -4, "Season_Snowbird": -4, "Season_ShoulderSeason": -2, "Season_SkiSeason": -8, "Season_ParkSeason": -6, "Season_CruiseSeason": -6, "Season_Weekender/SecondHomes": -4, }

REQUIRED_T100_COLS = {
    "ORIGIN_CITY_NAME",
    "DEST_CITY_NAME",
    "MONTH"
}

def _strip_accents(s: str) -> str:
    try:
        import unicodedata
        return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    except Exception:
        return s

def slug_city_state(s: str) -> str:
    """
    Regex-style normalization used for matching:
    - lowercase, strip accents
    - remove all non [a-z0-9]
    This lets 'Austin, TX' match 'austin, tx' regardless of spaces/special chars.
    """
    if not isinstance(s, str):
        return ""
    s = _strip_accents(s.strip().lower())
    return re.sub(r"[^a-z0-9]", "", s)

def find_cols(df, targets):
    cols = {c.lower(): c for c in df.columns}
    for t in targets:
        for lc, real in cols.items():
            if t in lc:
                return real
    return None

def ensure_col(df, col):
    if col not in df.columns:
        df[col] = 0

def enrich_overrides_from_text(cf: pd.DataFrame, city_col: str, why_col: str | None, primary_col: str | None):
    # Build combined lowercase text
    if (why_col and why_col in cf.columns) or (primary_col and primary_col in cf.columns):
        txt = pd.Series("", index=cf.index, dtype="object")
        if primary_col and primary_col in cf.columns:
            txt = txt.str.cat(cf[primary_col].astype(str), sep=" ")
        if why_col and why_col in cf.columns:
            txt = txt.str.cat(cf[why_col].astype(str), sep=" ")
        txt = txt.str.lower().fillna("")
    else:
        txt = pd.Series("", index=cf.index, dtype="object")

    def mark(col, cond):
        ensure_col(cf, col)
        cf[col] = np.where(cond | (cf[col].astype(int) == 1), 1, 0)

    # Non-capturing groups + na=False to avoid warnings
    mark("Market_ResortIsland",             txt.str.contains(r"\bresort\b|beach|coast|coastal|island", na=False))
    mark("Market_ParkGateway",              txt.str.contains(r"national park|park gateway", na=False))
    mark("Market_CruiseGateway/Port",       txt.str.contains(r"\bcruise\b", na=False))
    mark("Market_Ski/Outdoor_Gateway",      txt.str.contains(r"\bski", na=False))
    mark("Market_Military/Defense_Base",    txt.str.contains(r"\b(?:afb|nas|mcas|joint base|naval|air force|usaf|usmc|army|fort )\b", na=False))
    mark("Market_Military_Test/Eval_Range", txt.str.contains(r"(?:weapons station|flight test|fighter wing|test & evaluation)", na=False))
    mark("Market_StateCapital/Government",  txt.str.contains(r"(?:state capital|legislat)", na=False))
    mark("Market_NorthSlope_Oilfield",      txt.str.contains(r"(?:north slope|pipeline|pads|camps)", na=False))
    mark("Market_Mining",                   txt.str.contains(r"\b(?:mine|mining)", na=False))
    mark("Market_Remote/EAS",               txt.str.contains(r"\bremote\b|essential air service|\beas\b", na=False))
    mark("Market_Fishing/Cannery",          txt.str.contains(r"(?:fishing|cannery|seafood|tender)", na=False))
    mark("Market_AlaskaRegionalHub",        txt.str.contains(r"(?:alaska regional hub)", na=False))
    mark("Market_DiversifiedMetro/Corporate", txt.str.contains(r"(?:diversified metro|corporate|manufacturing|logistics|services base)", na=False))
    mark("Market_RegionalServices/Healthcare/LightIndustry", txt.str.contains(r"(?:regional services|healthcare|light industry)", na=False))
    mark("Market_Bizjet_Hub",               txt.str.contains(r"(?:bizjet|business aviation|private jet|fbo)", na=False))
    mark("Market_Border/Port_Logistics",    txt.str.contains(r"(?:border|port logistics)", na=False))
    mark("Market_Port/Shipbuilding/Shipyards", txt.str.contains(r"(?:shipbuilding|shipyard|deepwater port|port economy)", na=False))
    mark("Market_CoastalTourism/Golf/Aquarium", txt.str.contains(r"(?:coastal leisure|golf|aquarium)", na=False))
    mark("Market_Wine/Culinary",            txt.str.contains(r"(?:wine|culinary|tasting)", na=False))
    mark("Market_Weekender/HistoricTown",   txt.str.contains(r"(?:weekender|historic town|heritage)", na=False))

    # Seasons
    mark("Season_ParkSeason",               txt.str.contains(r"national park|park gateway", na=False))
    mark("Season_CruiseSeason",             txt.str.contains(r"\bcruise\b", na=False))
    mark("Season_SkiSeason",                txt.str.contains(r"\bski", na=False))
    mark("Season_SummerPeaks",              txt.str.contains(r"(?:summer|resort|beach|coast|coastal|island|strong weekend/seasonal spikes)", na=False))
    mark("Season_ShoulderSeason",           txt.str.contains(r"(?:shoulder-season|foliage|spring/fall)", na=False))
    mark("Season_Snowbird",                 txt.str.contains(r"snowbird", na=False))

    # University
    uni_any = txt.str.contains(r"\buniversity\b|college|athletics|home game|graduation", na=False)
    canonical_uni = "Season_University(HomeGames/Graduations/TermWeekdays)"
    ensure_col(cf, canonical_uni)
    cf[canonical_uni] = np.where(uni_any | (cf[canonical_uni].astype(int) == 1), 1, 0)

    # Conventions / legislative / quarter-end
    mark("Season_Conventions/Legislative/QuarterEnd", txt.str.contains(r"(?:convention|legislat|capital)", na=False))

def load_classification(path: Path):
    cf = pd.read_csv(path)

    # NEW: expect CITY_STATE present and use it for exact (normalized) matching
    city_state_col = find_cols(cf, ["city_state"]) or "CITY_STATE"

    # Figure out BTI/Confidence
    bti_col = next((c for c in cf.columns if "bti" in c.lower() or ("business" in c.lower() and "%" in c.lower())), None)
    if bti_col is None:
        raise ValueError("BTI column not found in classification file.")
    conf_col = next((c for c in cf.columns if "confidence" in c.lower()), None)
    if conf_col is None:
        raise ValueError("Confidence column not found in classification file.")

    # Optional text enrichment (uses Primary orientation / Why if present)
    why_col     = find_cols(cf, ["why"])
    primary_col = find_cols(cf, ["primary orientation"])
    enrich_overrides_from_text(cf, city_col=city_state_col, why_col=why_col, primary_col=primary_col)

    season_cols = [c for c in cf.columns if str(c).startswith("Season_")]
    market_cols = [c for c in cf.columns if str(c).startswith("Market_")]

    keep = [city_state_col, bti_col, conf_col] + season_cols + market_cols
    cf = cf[keep].copy()

    # Build normalized join key from CITY_STATE
    cf["CITYSTATE_SLUG"] = cf[city_state_col].astype(str).apply(slug_city_state)

    # Dedupe by slug, keep highest confidence
    cf["_CONF_NUM"] = pd.to_numeric(cf[conf_col], errors="coerce").fillna(0)
    cf = (
        cf.sort_values(["CITYSTATE_SLUG", "_CONF_NUM"], ascending=[True, False])
          .drop_duplicates("CITYSTATE_SLUG", keep="first")
          .drop(columns="_CONF_NUM")
    )

    # Cast numerics
    cf["BTI"]  = pd.to_numeric(cf[bti_col], errors="coerce")
    cf["CONF"] = pd.to_numeric(cf[conf_col], errors="coerce")
    for c in season_cols + market_cols:
        if c in cf.columns:
            cf[c] = pd.to_numeric(cf[c].replace("None", np.nan), errors="coerce").fillna(0).astype(np.int8)

    out_cols = ["CITYSTATE_SLUG","BTI","CONF"] + season_cols + market_cols
    return cf[out_cols], season_cols, market_cols

def compute_tpi_city_state(t100: pd.DataFrame, class_df: pd.DataFrame, season_cols, market_cols):
    """
    CITY_STATE-based matching:
      - Build slugs from ORIGIN_CITY_NAME / DEST_CITY_NAME (strip non-alnum, lowercase)
      - Join to classification on CITYSTATE_SLUG
      - Compute TPI only for rows where both endpoints match
      - Return (good_rows, missing_rows_with_reason)
    """
    t = t100.copy()

    # Slugs from City, ST strings (ignore spaces/special chars)
    t["ORIG_CITYSTATE_SLUG"] = t["ORIGIN_CITY_NAME"].astype(str).apply(slug_city_state)
    t["DEST_CITYSTATE_SLUG"] = t["DEST_CITY_NAME"].astype(str).apply(slug_city_state)

    base_cols = ["CITYSTATE_SLUG","BTI","CONF"] + season_cols + market_cols
    cf_o = class_df[base_cols].rename(columns={c: f"ORIG_{c}" for c in base_cols if c != "CITYSTATE_SLUG"})
    cf_d = class_df[base_cols].rename(columns={c: f"DEST_{c}" for c in base_cols if c != "CITYSTATE_SLUG"})

    t = t.merge(cf_o, left_on="ORIG_CITYSTATE_SLUG", right_on="CITYSTATE_SLUG", how="left").drop(columns=["CITYSTATE_SLUG"])
    t = t.merge(cf_d, left_on="DEST_CITYSTATE_SLUG", right_on="CITYSTATE_SLUG", how="left").drop(columns=["CITYSTATE_SLUG"])

    # Match status BEFORE fills
    orig_matched = t["ORIG_BTI"].notna() & t["ORIG_CONF"].notna()
    dest_matched = t["DEST_BTI"].notna() & t["DEST_CONF"].notna()
    can_assign = orig_matched & dest_matched

    # Coerce numerics
    for c in ["ORIG_BTI","ORIG_CONF","DEST_BTI","DEST_CONF","MONTH"]:
        t[c] = pd.to_numeric(t.get(c), errors="coerce")

    # Subset we can compute on
    work = t.loc[can_assign].copy()
    work[["ORIG_BTI","DEST_BTI"]] = work[["ORIG_BTI","DEST_BTI"]].fillna(50.0)
    work[["ORIG_CONF","DEST_CONF"]] = work[["ORIG_CONF","DEST_CONF"]].fillna(50.0)

    # --- Baseline (destination-biased, confidence-weighted) ---
    wA, wB = 1.0 - DESTINATION_WEIGHT, DESTINATION_WEIGHT
    denom = wA * work["ORIG_CONF"] + wB * work["DEST_CONF"]
    baseline = (wA * work["ORIG_CONF"] * work["ORIG_BTI"] + wB * work["DEST_CONF"] * work["DEST_BTI"]) / denom.replace(0, np.nan)
    baseline = baseline.fillna((work["ORIG_BTI"] + work["DEST_BTI"]) / 2.0)

    # --- Market deltas ---
    market_list = [m for m in MARKET_DELTAS if (f"ORIG_{m}" in work.columns or f"DEST_{m}" in work.columns)]
    if market_list:
        w_mkt = np.array([MARKET_DELTAS[m] for m in market_list], dtype=np.float32)
        o_cols = [work.get(f"ORIG_{m}", 0).astype(np.float32).to_numpy() for m in market_list]
        d_cols = [work.get(f"DEST_{m}", 0).astype(np.float32).to_numpy() for m in market_list]
        o_mat = np.stack(o_cols, axis=1) if o_cols else np.zeros((len(work),0), dtype=np.float32)
        d_mat = np.stack(d_cols, axis=1) if d_cols else np.zeros((len(work),0), dtype=np.float32)
        market_delta = ((o_mat @ w_mkt) * ORIG_MULT + (d_mat @ w_mkt) * DEST_MULT) if o_mat.shape[1] > 0 else np.zeros(len(work), dtype=np.float32)
        market_delta = pd.Series(market_delta, index=work.index, dtype="float32")
    else:
        market_delta = pd.Series(np.zeros(len(work), dtype=np.float32), index=work.index)

    # --- Lexical negatives (cap) ---
    lex_keys = [k for k in LEXICAL_NEG_DELTAS if (f"ORIG_{k}" in work.columns or f"DEST_{k}" in work.columns)]
    if lex_keys:
        w_lex = np.array([LEXICAL_NEG_DELTAS[k] for k in lex_keys], dtype=np.float32)
        olex = np.stack([work.get(f"ORIG_{k}", 0).astype(np.float32).to_numpy() for k in lex_keys], axis=1)
        dlex = np.stack([work.get(f"DEST_{k}", 0).astype(np.float32).to_numpy() for k in lex_keys], axis=1)
        lex_total = (olex @ w_lex) * (ORIG_MULT * 0.8) + (dlex @ w_lex) * DEST_MULT
        lex_total = np.maximum(lex_total, LEXICAL_CAP).astype(np.float32)
        lex_delta = pd.Series(lex_total, index=work.index, dtype="float32")
    else:
        lex_delta = pd.Series(np.zeros(len(work), dtype=np.float32), index=work.index)

    # --- Seasonality (month-aware) ---
    m = work["MONTH"].astype(int).to_numpy()
    ctx_peak     = np.isin(m, (6,7,8))
    ctx_shoulder = np.isin(m, (4,5,9,10))
    ctx_ski      = np.isin(m, (12,1,2,3))
    ctx_park     = np.isin(m, (6,7,8))
    ctx_cruise   = np.isin(m, (5,6,7,8,9))
    ctx_snowbird = np.isin(m, (12,1,2,3))
    ctx_quarter  = np.isin(m, (3,6,9,12))
    ctx_uni      = np.isin(m, (5,6,9,10,11))

    def season_component(flag, active_mask, base_delta):
        o = work.get(f"ORIG_{flag}", 0).astype(np.float32).to_numpy()
        d = work.get(f"DEST_{flag}", 0).astype(np.float32).to_numpy()
        delta = (o * ORIG_MULT + d * DEST_MULT * DEST_SEASON_BONUS) * base_delta
        delta[~active_mask] = 0.0
        return pd.Series(delta, index=work.index, dtype="float32")

    season_delta = pd.Series(np.zeros(len(work), dtype=np.float32), index=work.index)
    for flag, (mask, base) in {
        "Season_SummerPeaks":   (ctx_peak,     SEASON_DELTAS["Season_SummerPeaks"]),
        "Season_ShoulderSeason":(ctx_shoulder, SEASON_DELTAS["Season_ShoulderSeason"]),
        "Season_SkiSeason":     (ctx_ski,      SEASON_DELTAS["Season_SkiSeason"]),
        "Season_ParkSeason":    (ctx_park,     SEASON_DELTAS["Season_ParkSeason"]),
        "Season_CruiseSeason":  (ctx_cruise,   SEASON_DELTAS["Season_CruiseSeason"]),
        "Season_Snowbird":      (ctx_snowbird, SEASON_DELTAS["Season_Snowbird"]),
    }.items():
        season_delta = season_delta.add(season_component(flag, mask, base), fill_value=0.0)

    # University & Conventions
    uni_flag = "Season_University(HomeGames/Graduations/TermWeekdays)"
    if f"ORIG_{uni_flag}" in work.columns or f"DEST_{uni_flag}" in work.columns:
        season_delta = season_delta.add(season_component(uni_flag, ctx_uni, -6.0), fill_value=0.0)

    conv_flag = "Season_Conventions/Legislative/QuarterEnd"
    if f"ORIG_{conv_flag}" in work.columns or f"DEST_{conv_flag}" in work.columns:
        season_delta = season_delta.add(season_component(conv_flag, ctx_quarter, +5.0), fill_value=0.0)

    # --- Confidence-scaled total adjustment ---
    s_airport = (work["ORIG_CONF"] + work["DEST_CONF"]) / 200.0
    total_adj = (market_delta + lex_delta + season_delta) * s_airport.astype("float32")

    # --- Final TPI, label, confidence ---
    tpi = (baseline + total_adj).clip(0.0, 100.0)
    low, high = THRESHOLDS
    label = np.where(tpi >= high, "Business-heavy",
                     np.where(tpi <= low, "Leisure-heavy", "Mixed"))
    b = np.sqrt((work["ORIG_CONF"]/100.0) * (work["DEST_CONF"]/100.0))
    half_gap = (high - low) / 2.0
    center = (high + low) / 2.0
    mgn = np.where(label=="Business-heavy", np.minimum(1.0, (tpi - high)/half_gap),
          np.where(label=="Leisure-heavy", np.minimum(1.0, (low - tpi)/half_gap),
                   1.0 - np.minimum(1.0, np.abs(tpi - center)/half_gap)))
    w_air, w_m = CONF_WEIGHTS
    conf = np.clip(100.0 * (w_air * b + w_m * mgn), 0.0, 100.0)

    # --- Prepare outputs in 't' and assign back safely ---
    t["TPI"] = pd.Series(np.nan, index=t.index, dtype="float32")
    t["TPI_Label"] = pd.Series(pd.NA, index=t.index, dtype="object")
    t["TPI_Confidence"] = pd.Series(np.nan, index=t.index, dtype="float32")

    work["TPI"] = tpi.astype("float32")
    work["TPI_Label"] = pd.Series(label, index=work.index, dtype="object")
    work["TPI_Confidence"] = conf.astype("float32")

    t.loc[work.index, "TPI"] = work["TPI"].to_numpy()
    t.loc[work.index, "TPI_Label"] = work["TPI_Label"].astype("object").to_numpy()
    t.loc[work.index, "TPI_Confidence"] = work["TPI_Confidence"].to_numpy()

    # --- Missing diagnostics (ALIGN MASKS TO missing.index) ---
    missing_mask = ~can_assign
    missing = t.loc[missing_mask].copy()

    om_miss = (~orig_matched &  dest_matched).loc[missing.index].to_numpy()
    dm_miss = ( orig_matched & ~dest_matched).loc[missing.index].to_numpy()
    bm_miss = (~orig_matched & ~dest_matched).loc[missing.index].to_numpy()

    missing["MISSING_REASON"] = np.select(
        [om_miss, dm_miss, bm_miss],
        ["Origin not in classification (CITY_STATE slug)",
         "Destination not in classification (CITY_STATE slug)",
         "Both origin and destination not in classification (CITY_STATE slug)"],
        default="Unknown"
    )

    # --- Return both dataframes ---
    good = t.loc[~missing_mask].copy()
    cols = list(t100.columns) + ["TPI","TPI_Label","TPI_Confidence"]
    missing_cols = [c for c in list(t100.columns) + ["MISSING_REASON","ORIG_CITYSTATE_SLUG","DEST_CITYSTATE_SLUG","TPI","TPI_Label","TPI_Confidence"] if c in missing.columns]
    return good[cols], missing[missing_cols]


def _count_csv_rows(path: Path) -> int:
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        total = -1
        for total, _ in enumerate(f, start=0):
            pass
    return max(0, total)

def main():
    # Validate required columns (state columns no longer required for join)
    hdr = pd.read_csv(T100_PATH, nrows=0)
    required = {"ORIGIN_CITY_NAME","DEST_CITY_NAME","MONTH"}
    missing_req = required - set(hdr.columns)
    if missing_req:
        raise ValueError(f"T100 file is missing required columns: {sorted(missing_req)}")

    start = time.time()
    class_df, season_cols, market_cols = load_classification(CLASS_PATH)

    # Fresh outputs
    for p in [OUT_PATH, MISSING_OUT, UNMATCHED_ORIG_COUNTS, UNMATCHED_DEST_COUNTS]:
        if p.exists(): p.unlink()

    total_rows = _count_csv_rows(T100_PATH)
    print(f"[init] Input: {T100_PATH.name}  | rows ≈ {total_rows:,}")
    print(f"[init] Classification: {CLASS_PATH.name}  | cities: {len(class_df):,}")
    print(f"[init] Outputs: {OUT_PATH.name}, {MISSING_OUT.name}")
    print("-"*80, flush=True)

    processed = 0
    batch_idx = 0
    header_written = False
    missing_header_written = False

    # Diagnostics counters
    unmatched_rows_total = 0
    orig_counter = Counter()
    dest_counter = Counter()

    for chunk in pd.read_csv(T100_PATH, chunksize=CHUNK_SIZE):
        batch_idx += 1
        t0 = time.time()

        good, missing_df = compute_tpi_city_state(chunk, class_df, season_cols, market_cols)

        # Append good rows
        good.to_csv(OUT_PATH, index=False, mode="a", header=not header_written)
        header_written = True

        # Append missing rows + update diagnostics
        if not missing_df.empty:
            missing_df.to_csv(MISSING_OUT, index=False, mode="a", header=not missing_header_written)
            missing_header_written = True

            unmatched_rows_total += len(missing_df)

            # Update city/state counters from the raw T100 names
            if "ORIGIN_CITY_NAME" in missing_df.columns:
                for name in missing_df["ORIGIN_CITY_NAME"]:
                    if pd.notna(name):
                        # derive (City, ST) pair by simple split on last comma for diagnostics
                        parts = str(name).rsplit(",", 1)
                        city = parts[0].strip()
                        st = parts[1].strip() if len(parts) == 2 else ""
                        orig_counter[(city, st)] += 1
            if "DEST_CITY_NAME" in missing_df.columns:
                for name in missing_df["DEST_CITY_NAME"]:
                    if pd.notna(name):
                        parts = str(name).rsplit(",", 1)
                        city = parts[0].strip()
                        st = parts[1].strip() if len(parts) == 2 else ""
                        dest_counter[(city, st)] += 1

        processed += len(chunk)
        elapsed = time.time() - t0
        pct = (processed / total_rows * 100.0) if total_rows else 0.0
        out_mb = OUT_PATH.stat().st_size / (1024*1024)
        miss_mb = (MISSING_OUT.stat().st_size / (1024*1024)) if MISSING_OUT.exists() else 0.0
        print(f"[batch {batch_idx:02d}] {len(chunk):,} rows in {elapsed:0.2f}s | "
              f"total {processed:,}/{total_rows:,} ({pct:0.1f}%) | "
              f"out: {OUT_PATH.name} ({out_mb:0.2f} MB) | missing: {miss_mb:0.2f} MB", flush=True)

    # Write unmatched city/state counts
    if orig_counter:
        df_orig = (pd.DataFrame([(c, s, n) for (c, s), n in orig_counter.items()],
                                columns=["Origin_City","Origin_State","Unmatched_Row_Count"])
                   .sort_values(["Unmatched_Row_Count","Origin_State","Origin_City"], ascending=[False, True, True]))
        df_orig.to_csv(UNMATCHED_ORIG_COUNTS, index=False)
    else:
        pd.DataFrame(columns=["Origin_City","Origin_State","Unmatched_Row_Count"]).to_csv(UNMATCHED_ORIG_COUNTS, index=False)

    if dest_counter:
        df_dest = (pd.DataFrame([(c, s, n) for (c, s), n in dest_counter.items()],
                                columns=["Dest_City","Dest_State","Unmatched_Row_Count"])
                   .sort_values(["Unmatched_Row_Count","Dest_State","Dest_City"], ascending=[False, True, True]))
        df_dest.to_csv(UNMATCHED_DEST_COUNTS, index=False)
    else:
        pd.DataFrame(columns=["Dest_City","Dest_State","Unmatched_Row_Count"]).to_csv(UNMATCHED_DEST_COUNTS, index=False)

    total_elapsed = time.time() - start
    out_mb = OUT_PATH.stat().st_size / (1024*1024)
    miss_mb = (MISSING_OUT.stat().st_size / (1024*1024)) if MISSING_OUT.exists() else 0.0
    print("-"*80)
    print(f"[done] Wrote GOOD rows: {OUT_PATH}  ({out_mb:0.2f} MB)")
    print(f"[done] Wrote MISSING rows: {MISSING_OUT}  ({miss_mb:0.2f} MB)")
    print(f"[done] Total input rows: {processed:,}  | batches: {batch_idx}")
    print(f"[done] Unmatched rows (cannot assign TPI via CITY_STATE slug): {unmatched_rows_total:,}")
    print(f"[done] Unique unmatched Origin city/state: {len(orig_counter):,}  | saved -> {UNMATCHED_ORIG_COUNTS.name}")
    print(f"[done] Unique unmatched Dest city/state:   {len(dest_counter):,}  | saved -> {UNMATCHED_DEST_COUNTS.name}")

if __name__ == "__main__":
    main()


[init] Input: consolidated_t100_data_cleaned.csv  | rows ≈ 815,641
[init] Classification: city_business_leisure_classification_detailed.csv  | cities: 1,629
[init] Outputs: consolidated_t100_with_tpi_city_state.csv, missing_consolidated_t100_with_tpi_city_state.csv
--------------------------------------------------------------------------------
[batch 01] 250,000 rows in 12.49s | total 250,000/815,641 (30.7%) | out: consolidated_t100_with_tpi_city_state.csv (70.68 MB) | missing: 0.00 MB
[batch 02] 250,000 rows in 11.81s | total 500,000/815,641 (61.3%) | out: consolidated_t100_with_tpi_city_state.csv (141.52 MB) | missing: 0.00 MB
[batch 03] 250,000 rows in 10.56s | total 750,000/815,641 (92.0%) | out: consolidated_t100_with_tpi_city_state.csv (211.96 MB) | missing: 0.00 MB
[batch 04] 65,641 rows in 3.77s | total 815,641/815,641 (100.0%) | out: consolidated_t100_with_tpi_city_state.csv (230.37 MB) | missing: 0.00 MB
-----------------------------------------------------------------------